In [1]:
import pandas as pd
import numpy as np
import glob
import re
import os

# Underlyings

* Merge and Clean Data. Calculate 10-D MA and Daily Growth

In [2]:
Project_Dir = os.getcwd()
Raw_Data_Dir = Project_Dir + r"/Raw_Data"
Processed_Data_Dir = Project_Dir + r"/Processed_Data"
csv_raw_ls = glob.glob(Raw_Data_Dir + r'/*.csv')

In [3]:
DATA_STACK = []
for f in csv_raw_ls:
    tmp = pd.read_csv(f, usecols=['Date', 'Adj Close'], parse_dates=['Date'])
    tmp.columns=['Date', 'Close']
    tmp['Fig'] = re.search('[A-Z]{3,4}.csv', f).group(0).replace(".csv","")
    tmp['Ten_ma'] = tmp.Close.rolling(10, min_periods=1).mean()
    tmp['pct_chg'] = tmp.Close.pct_change()
    DATA_STACK.append(tmp.dropna())
DATA_STACK = pd.concat(DATA_STACK)

In [4]:
DATA_STACK.head()

,Date,Close,Fig,Ten_ma,pct_chg
1,2013-11-27,75.842369,SCC,75.862244,-0.000524
2,2013-11-29,75.246117,SCC,75.656868,-0.007862
3,2013-12-02,75.802612,SCC,75.693304,0.007396
4,2013-12-03,77.313110,SCC,76.017265,0.019927
5,2013-12-04,77.909355,SCC,76.332613,0.007712


* Check the Trading Span of each Underlying, and get rid of ones with short span. Keep only the Established ones with longer trading history

In [5]:
dt_trade_span=DATA_STACK.groupby('Fig')\
    .agg({'Date':[np.min, np.max,lambda x: (np.max(x)-np.min(x))]})\
    .sort_values([('Date','<lambda>')])

In [6]:
dt_trade_span

Date                     
           amin       amax  <lambda>
Fig                                 
MEXX 2017-05-04 2018-11-23  568 days
VMIN 2016-05-04 2018-11-23  933 days
NAIL 2015-08-20 2018-11-23 1191 days
GUSH 2015-06-01 2018-11-23 1271 days
HOML 2015-03-11 2018-11-23 1353 days
AMZA 2014-10-03 2018-11-23 1512 days
FNG  2014-07-14 2018-11-23 1593 days
ERX  2013-11-27 2018-11-23 1822 days
GASL 2013-11-27 2018-11-23 1822 days
OIH  2013-11-27 2018-11-23 1822 days
SCC  2013-11-27 2018-11-23 1822 days
SOXL 2013-11-27 2018-11-23 1822 days
USD  2013-11-27 2018-11-23 1822 days

In [7]:
data_estbsed=DATA_STACK[~DATA_STACK.Fig.isin(['VMIN','MEXX'])& (DATA_STACK.Date>='2015-08-20')]

* Calculate 104 Weekly Range to compare with current price. 10-D MA Metric.

In [8]:
(data_estbsed.Date.dt.strftime("%y-%W")).sort_values(ascending=False).unique()[0:104]

array(['18-47', '18-46', '18-45', '18-44', '18-43', '18-42', '18-41',
       '18-40', '18-39', '18-38', '18-37', '18-36', '18-35', '18-34',
       '18-33', '18-32', '18-31', '18-30', '18-29', '18-28', '18-27',
       '18-26', '18-25', '18-24', '18-23', '18-22', '18-21', '18-20',
       '18-19', '18-18', '18-17', '18-16', '18-15', '18-14', '18-13',
       '18-12', '18-11', '18-10', '18-09', '18-08', '18-07', '18-06',
       '18-05', '18-04', '18-03', '18-02', '18-01', '17-52', '17-51',
       '17-50', '17-49', '17-48', '17-47', '17-46', '17-45', '17-44',
       '17-43', '17-42', '17-41', '17-40', '17-39', '17-38', '17-37',
       '17-36', '17-35', '17-34', '17-33', '17-32', '17-31', '17-30',
       '17-29', '17-28', '17-27', '17-26', '17-25', '17-24', '17-23',
       '17-22', '17-21', '17-20', '17-19', '17-18', '17-17', '17-16',
       '17-15', '17-14', '17-13', '17-12', '17-11', '17-10', '17-09',
       '17-08', '17-07', '17-06', '17-05', '17-04', '17-03', '17-02',
       '17-01', '16-

In [9]:
dt_week_104=data_estbsed[data_estbsed.Date.dt.strftime("%y-%W")>='16-48']
dt_range_104=[]
for fig in data_estbsed.Fig.unique():
    dt_tmp=dt_week_104.loc[dt_week_104.Fig==fig,:]
    crt=dt_tmp.loc[dt_tmp.Date.idxmax,'Ten_ma']
    low,high=[np.min(dt_tmp.Ten_ma),np.max(dt_tmp.Ten_ma)]
    pos=(crt-low)/(high-low)
    dt_range_104.append([fig,crt,low,high,pos])


In [10]:
dt_range_104=pd.DataFrame(dt_range_104,
             columns=['Fig','Current','104_Low',
                      '104_High','Position']
            ).sort_values('Position').set_index('Fig').round(2)

In [11]:
dt_range_104

,Current,104_Low,104_High,Position
Fig,,,,
GASL,11.67,11.67,51.31,0.00
OIH,19.09,19.09,33.50,0.00
GUSH,18.49,15.87,107.67,0.03
NAIL,27.82,24.97,105.11,0.04
AMZA,6.17,6.06,7.88,0.06
HOML,25.87,22.65,67.25,0.07
SCC,19.06,16.94,36.25,0.11
ERX,24.03,21.72,41.93,0.11
SOXL,96.96,51.16,188.69,0.33


* Go and Find Options Data based on 104 Week Position

In [12]:
dt_range_104[dt_range_104.Position<0.2].index

Index(['GASL', 'OIH', 'GUSH', 'NAIL', 'AMZA', 'HOML', 'SCC', 'ERX'], dtype='object', name='Fig')

# Option Data

* Cleanse

In [13]:
DATA_OPTION=pd.read_csv(Raw_Data_Dir+'/Options/Option_Dec_1.csv',parse_dates=[0],
                        usecols=['Root','Ask','Strike','Puts'])

In [14]:
DATA_OPTION.columns=['Maturity','Fig','Strike','Ask_p']

In [15]:
DATA_OPTION.head(10)

,Maturity,Fig,Strike,Ask_p
0,2018-12-21,GASL,1.0,0.55
1,2018-12-21,GASL,2.0,0.55
2,2018-12-21,GASL,3.0,0.55
3,2018-12-21,GASL,4.0,0.55
4,2018-12-21,GASL,5.0,0.75
5,2018-12-21,GASL,6.0,0.15
6,2018-12-21,GASL,7.0,0.25
7,2018-12-21,GASL,8.0,0.45
8,2018-12-21,GASL,9.0,0.80
9,2019-01-18,GASL,2.0,4.90


In [16]:
DATA_OPTION['Fig'].unique()

array(['GASL', 'OIH', 'GUSH', 'AMZA', 'ERX', 'SOXL', 'USD'], dtype=object)

# Process and Save Option & Underlying

* Get Underlying that trade Option. Save and Validation

In [17]:
dt_trade_span

Date                     
           amin       amax  <lambda>
Fig                                 
MEXX 2017-05-04 2018-11-23  568 days
VMIN 2016-05-04 2018-11-23  933 days
NAIL 2015-08-20 2018-11-23 1191 days
GUSH 2015-06-01 2018-11-23 1271 days
HOML 2015-03-11 2018-11-23 1353 days
AMZA 2014-10-03 2018-11-23 1512 days
FNG  2014-07-14 2018-11-23 1593 days
ERX  2013-11-27 2018-11-23 1822 days
GASL 2013-11-27 2018-11-23 1822 days
OIH  2013-11-27 2018-11-23 1822 days
SCC  2013-11-27 2018-11-23 1822 days
SOXL 2013-11-27 2018-11-23 1822 days
USD  2013-11-27 2018-11-23 1822 days

In [18]:
dt_option=DATA_OPTION.join(dt_trade_span.Date,on='Fig')

In [19]:
dt_option['D2M']=dt_option.Maturity-dt_option.amax
dt_option.drop(['amin','amax','<lambda>'],axis=1,inplace=True)

In [20]:
dt_option.head()

,Maturity,Fig,Strike,Ask_p,D2M
0,2018-12-21,GASL,1.0,0.55,28 days
1,2018-12-21,GASL,2.0,0.55,28 days
2,2018-12-21,GASL,3.0,0.55,28 days
3,2018-12-21,GASL,4.0,0.55,28 days
4,2018-12-21,GASL,5.0,0.75,28 days


In [21]:
dt_option.shape

(453, 5)

In [22]:
np.sort(dt_option.D2M.dt.days.unique())

array([ 14,  21,  25,  28,  35,  42,  49,  56,  84, 112, 146, 175, 210,
       420, 784])

In [23]:
dt_opt_near=dt_option[(dt_option.D2M.dt.days<=120)]

In [24]:
dt_opt_near.shape

(408, 5)

* Backwards 2nd time screening and Finalize Save

In [25]:
final_col_ls=[col for col in dt_opt_near.Fig.unique() if col not in ['SOXL']]
final_col_ls

['GASL', 'OIH', 'GUSH', 'AMZA', 'ERX', 'USD']

In [26]:
dt_option[dt_option.Fig.isin(final_col_ls)].to_csv(Processed_Data_Dir+'/Options.csv')

In [27]:
data_estbsed[data_estbsed.Fig.isin(final_col_ls)].to_csv(Processed_Data_Dir+'/Underlyings.csv')